# DSPy

## What?
Framework to algorithmically optimizing LM prompts and weights.

## Why?
Using LMs to build a complex system we generally have to: 
- Break the problem down into steps
- Prompt your LM well until each step works well in isolation
- Tweak the steps to work well together
- Generate synthetic examples to tune each step
- Use these examples to finetune smaller LMs to cut costs.

Currently, this is hard and messy: every time you change your pipeline, your LM, or your data, all prompts (or finetuning steps) may need to change.

## How?
DSPy breaks this process into following three abstractions:
- **Signatures:** Abstract the input and output behaviour
- **Modules:** Defines the flow of your program and sets up a pipeline. E.g of modules: `Predict`, `ChainOfThought`, `ProgramOfThought`, `MultiChainComparison` and `React`.
- **Optimizers:** Also known as `Teleprompters`, it takes the program, a training set and an evaluation metric and returns a new optimized program for the required use-case. Used to train smaller LMs (student) using larger LMs (teacher).

> Training Set can be small or have incomplete examples or without labels unless needed to be used in metric.
Metric can be `Exact Match (EM)` or `F1` or any custom defined metric

## DSPy Architecture

![dspy_arch](./assets/dspy_arch.png)

### Signature:
Use signature to tell, `what to do`, instead of `how to do`. Need not write huge prompts.
DSPy supports inline short strings as signatures, can always write custom classes for the same.

Some signatures available in DSPy:

| Task                        | Signature                      |
|-----------------------------|--------------------------------|
| Question-Answering         | "question -> answer"           |
| Summarization               | "document -> summary"          |
| Sentiment classification    | "sentence -> sentiment"        |
| RAG                         | "context, question -> answer"  |
| MCQs with Reasoning        | "question, choices -> reasoning, selection" |


### Module:
Takes the signature and converts it into a sophisticated prompt, based on a given technique and LLM used. Can be thought of as a model layer defined in Pytorch that learns from data (input/output).

### Optimizer:
DSPy optimizer can optimize 3 things
- LM weights
- Instructions (Prompt/Signature)
- Demonstrations of Input-Ouput Behaviour

Current available optimizers: [https://dspy-docs.vercel.app/docs/building-blocks/optimizers#what-dspy-optimizers-are-currently-available](https://dspy-docs.vercel.app/docs/building-blocks/optimizers#what-dspy-optimizers-are-currently-available)

#### Which optimizer should I use?
As a rule of thumb, if you don't know where to start, use `BootstrapFewShotWithRandomSearch`.

Here's the general guidance on getting started:
- If you have `very little data`, e.g. 10 examples of your task, use `BootstrapFewShot`.
- If you have `slightly more data`, e.g. 50 examples of your task, use `BootstrapFewShotWithRandomSearch`.
- If you have `more data than that`, e.g. 300 examples or more, use `MIPRO`.
- If you have been able to use one of these with a `large LM` (e.g., 7B parameters or above) and need a very efficient program, `compile` that down to a `small LM` with `BootstrapFinetune`.

## General Workflow
Whatever the task, the general workflow is:

- Collect a little bit of data.
- Define examples of the inputs and outputs of your program (e.g., questions and their answers). This could just be a handful of quick examples you wrote down. If large datasets exist, the more the merrier!
- Define the modules (i.e., sub-tasks) of your program and the way they should interact together to solve your task.
- Define some validation logic. What makes for a good run of your program? Maybe the answers need to have a certain length or stick to a particular format? Specify the logic that checks that.
- Compile! Ask DSPy to compile your program using your data. The compiler will use your data and validation logic to optimize your program (e.g., prompts and modules) so it's efficient and effective!Iterate.
- Repeat the process by improving your data, program, validation, or by using more advanced features of the DSPy compiler.

## Demo - RAG (Unoptimized)

We shall try RAG on truefoundry docs that are ingested in local docker based Qdrant deployment 

In [1]:
# First need the embedding
from embeddings.mixedbread import MixBreadEmbeddings
embedding_model = MixBreadEmbeddings(
    model_name="mixedbread-ai/mxbai-embed-large-v1"
)

In [2]:
# Set up Retriever
from vectordb.qdrant import CustomQdrantRetriever, QdrantClient

qdrant_client = QdrantClient(url="http://localhost:6333")
retriever = CustomQdrantRetriever(
    qdrant_collection_name="creditcard", 
    qdrant_client=qdrant_client, 
    embedding_model=embedding_model,
    k=5,
)


In [3]:
# also have other keys like metadata & score
retriever("What is creditcard", k=2)

  0%|          | 0/1 [00:00<?, ?it/s]

[{'long_text': '# [Regalia gold credit card](https://www.hdfcbank.com/personal/pay/cards/credit-cards/regalia-gold-credit-card)\n## Features\n#### Contactless Payment\nContactless Payment\nThe HDFC Bank Regalia Gold Credit Card is enabled for contactless payments, facilitating fast, convenient and secure payments at retail outlets.\n\\*To see if your Card is contactless, look for the contactless network symbol on your Card.\n(Please note that in India, payment through contactless mode is allowed for a maximum of ₹5000 in a single transaction without being asked to input your [Credit Card PIN](/personal/pay/cards/credit-cards/forgot-card-pin "/personal/pay/cards/credit-cards/forgot-card-pin"). However, if the amount is higher than or equal to ₹5000, the Cardholder has to enter the Credit Card PIN for security reasons.)'},
 {'long_text': '# [Infinia credit card](https://www.hdfcbank.com/personal/pay/cards/credit-cards/infinia-credit-card)\n## Features\n#### Insurance/Comprehensive protec

In [4]:
# Set up LLM
from dspy import OllamaLocal

llm = OllamaLocal(
    model="llama3:8b", 
    model_type="text", 
    max_tokens=1024, 
    top_p=1, 
    top_k=20, 
    temperature=0.1,
    base_url="http://localhost:11434", 
    frequency_penalty=0.9,
    presence_penalty=2
)

In [5]:
# Configure the settings
import dspy
dspy.configure(lm=llm, rm=retriever)

In [6]:
# Generate Signature for Input
class GenerateAnswer(dspy.Signature):
    """Answer the question in detail using the context."""

    context = dspy.InputField(desc="Contains relevant facts to answer the question")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="Detailed answer with respect to given question and context")

In [7]:
class RAG(dspy.Module):

    def __init__(self, k: int = 5):
        super().__init__()
        
        self.k = k
        self.retriever = dspy.Retrieve(k=self.k)
        self.generate_answer = dspy.ChainOfThought(signature=GenerateAnswer)

    def forward(self, question, k=None):
        passages = self.retriever(question, k).passages
        # print("Passages: ", passages)
        prediction = self.generate_answer(context=passages, question=question)
        return dspy.Prediction(context=passages, answer=prediction.answer)

In [8]:
# Ask any question you like to this simple RAG program.
my_question = "Explain different types of credit cards?"
uncompiled_rag = RAG(k=10)
# Get the prediction. This contains `pred.context` and `pred.answer`.
prediction = uncompiled_rag(my_question)

print(f"ANSWER: {prediction.answer}")
print("====================================\n\n")

for context in zip(prediction.context):
    print(f"Context: {context}")
    print("\n\n====================================")


  0%|          | 0/1 [00:00<?, ?it/s]

ANSWER: There are several types of credit cards offered by banks like HDFC Bank, each catering to different segments of customers with unique features and benefits.

Here's a breakdown:

* Infinia Credit Card (HDFC Bank): A premium card offering reward points redemption and cashback offers.
* Regalia Gold Credit Card (HDFC Bank): A high-end card providing smart EMI options for converting big spends into easy installments.
* Diners Club Black: A travel-focused credit card with revolving credit at a nominal interest rate, exclusive airport lounge access, and other benefits.

The HDFC Bank UPI RuPay Credit Card is specifically designed to provide:

1. Cashpoints:
	* 3% cashback on groceries, supermarkets & dining
	* 2% on utility spends (Maximum of ₹500 in a calendar month)
	* 1% on other transactions (excluding Rent, Wallet loads, EMI, Fuel, Insurance Payments & Government categories) (Maximum of ₹500 in a calendar month)

These cards cater to different customer segments:

* Infinia: For

In [9]:
# Track llm history
llm.inspect_history(n=1)





Answer the question in detail using the context.

---

Follow the following format.

Context: Contains relevant facts to answer the question

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: Detailed answer with respect to given question and context

---

Context:
[1] «# [Freedom card new](https://www.hdfcbank.com/personal/pay/cards/credit-cards/freedom-card-new)
## Features
#### Additional Features
Additional Features
**Zero lost Card liability :** In the unfortunate event of losing your HDFC Bank Freedom Credit Card, report it immediately to our 24-hour call centre. On reporting the loss immediately, you have zero liability on any fraudulent transactions made on your Credit Card.
**Interest Free Credit Period :** Avail up to 50 days of interest free period on your HDFC Bank Freedom Credit Card from the date of purchase (subject to the submission of the charge by the Merchant).
**Revolving Credit :** Enjoy Revolving Cred